Commit: [0edba49611125b58f5460ac362df2f5dc2e02e44 (message: "Revert to 6th of january")](https://github.com/squillero/computational-intelligence/tree/0edba49611125b58f5460ac362df2f5dc2e02e44)

In [ ]:
from tqdm.auto import tqdm
from abc import ABC, abstractmethod
from copy import deepcopy, copy
from enum import Enum

import matplotlib.pyplot as plt
import random
from random import choice

import numpy as np
import time
import pickle
import os

In [ ]:
EPISODES_TRAINING_FITNESS = 80
EPISODES_TRAINING = 110
EPISODES_GAME = 100
PERCENT_THRESHOLD = 0.6
TOLERANCE_EPOCHS = int(EPISODES_TRAINING ** PERCENT_THRESHOLD)
BOARD_SIZE = 5
NUM_OF_RULES = 14
TRAINING_FILE_NAME = 'lambda_20_training_70_training_fitness_80_rules_14_initialsigma_0.35_final_0.08_random.pkl'

# Rules

* All the comments are referred to our ESPlayer as first player ( index == 0)

In [ ]:
def r1(game: 'MyGame'):

    '''
      Check for a row without 0 in order to put 0 in that row
    '''

    board_situation = game.get_board()

    for row_index, row in enumerate(board_situation):
        result = np.all(row != game.current_player_idx)
        if result == True:
            if row_index >= 1 and row_index <= 3:
                if not (row[0] == (1 - game.current_player_idx) and row[4] == (1 - game.current_player_idx)):
                    return True
            else:
                return True

    return False



def r2(game: 'MyGame'):

    '''
      Check for a column without O  in order to put 0 in that column
    '''

    board = game.get_board()

    board = board.T # transposed
    for i, column in enumerate(board):
        result = np.all(column != game.current_player_idx)
        if result == True:
            if i>=1 and i<=3:
                if not (column[0]== (1 - game.current_player_idx) and column[4]== (1 -game.current_player_idx)):
                    return True
            else:
                return True

    return False


def r3(game: 'MyGame'):

    '''
      If we are doing 5 on row , on the perimeter
      of the board add cube with our face (if and only if the position that allow the 5-sequence is available)
    '''
    board = game.get_board()

    for row in board:
        count_agent = game.current_player_idx
        for el in row:
            if el == game.current_player_idx:
                count_agent += 1
                if count_agent >= 4 and (row[0] == -1 or row[4] == -1): # There are four consecutive 0 in a row and there is the row[0] or row[4] available ( == -1)
                    return True
            else:
                count_agent = 0


    return False


def r4(game: 'MyGame'):
    '''
      If we are doing 5 on one column , on the perimeter
      of the board add cube with our face (if and only if the position that allow the 5-sequence is available)
    '''
    board = game.get_board()

    for column in board.T:
        count_agent = 0
        for el in column:
            if el == game.current_player_idx:
                count_agent += 1
                if count_agent >= 4 and (column[0] == -1 or column[4] == -1): # There are four consecutive 0 in a column and there is the column[0] or column[4] available ( == -1)
                    return True
            else:
                count_agent = 0


    return False


def r5(game: 'MyGame'):

    '''
        If we are doing 5 on one of the two main diagonals , on the perimeter
        of the board add cube with our sign on the corner which complete the 5-sequence (if and only if that corner is available)
    '''
    board = game.get_board()

    diagonal_main = np.diagonal(board)
    diagonal_secondary = np.diagonal(board[:, ::-1]) # [:, ::-1] returns the matrix with inverted columns col == 4 become col == 0

    count_non_agent_main = np.count_nonzero(diagonal_main != game.current_player_idx)

    count_non_agent_secondary = np.count_nonzero(diagonal_secondary != game.current_player_idx)

    if (diagonal_main[0] == -1 or diagonal_main[4] == -1) and count_non_agent_main == 1:
      if diagonal_main[0] == -1 and (board[0,1] == game.current_player_idx or board[1,0] == game.current_player_idx):
        return True
      if diagonal_main[4] == -1 and (board[3,4] == game.current_player_idx or board[4,3] == game.current_player_idx):
        return True

    if (diagonal_secondary[0] == -1 or diagonal_secondary[4] == -1) and count_non_agent_secondary == 1:
      if diagonal_secondary[0] == -1 and (board[0,3] == game.current_player_idx or board[1,4] == game.current_player_idx):
        return True
      if diagonal_secondary[4] == -1 and (board[3,0] == game.current_player_idx or board[4,1] == game.current_player_idx):
        return True

    return False


def r6(game: 'MyGame'):

    '''
        Blocks the opponent's win: if the opponent is going to make 5 on the row,  block the winning move (if and only if
        at the extreme there are cubes with our face or without face)
    '''

    board = game.get_board()

    for row in board:
        count_opponent = 0
        for el in row:
            if el == 1 - game.current_player_idx:
                count_opponent += 1
                if count_opponent >= 4 and (row[0] == -1 or row[4] == -1): # There are four consecutive 1 in a row and there is the row[0] or row[4] available ( == -1)
                    return True
            else:
                count_opponent = 0

    return False


def r7(game: 'MyGame'):

    '''
        Blocks the opponent's win: if the opponent is going to make 5 on the col,  block the winning move (if and only if
        at the extreme there are cubes with our face or without face)

    '''

    board = game.get_board()

    for column in board.T:
        count_opponent = 0
        for el in column:
            if el ==  1 - game.current_player_idx:
                count_opponent += 1
                if count_opponent >= 4 and (column[0] == -1 or column[4] == -1): # There are four consecutive 1 in a column and there is the column[0] or column[4] available ( == -1)
                    return True
            else:
                count_opponent = 0


    return False

def r8(game: 'MyGame'):


    '''
        unoptimal move: if the opponent is going to make 5 on the row/column/main diagonal/secondary diagonal,  DO NOT block the winning move (if and only if
        at the extreme there are cubes with our face or without face)

    '''


    board = game.get_board()

    # row
    for row in board:
        count_opponent = 0
        for el in row:
            if el == 1 -  game.current_player_idx:
                count_opponent += 1
        if count_opponent >= 4 and (row[0] == -1 or row[4] == -1): # There are four consecutive 1 in a row and there is the row[0] or row[4] available ( == -1)
            return True

    # column
    for col in board.T:
        count_opponent = 0
        for el in col:
            if el == 1 - game.current_player_idx:
                count_opponent += 1
        if count_opponent >= 4 and (col[0] == -1 or col[4] == -1): # There are four consecutive 1 in a col and there is the row[0] or row[4] available ( == -1)
            return True

    # diagonal: main and secondary
    diagonal_main = np.diagonal(board)
    diagonal_secondary = np.diagonal(board[:, ::-1]) # [:, ::-1] returns the matrix with inverted columns col == 4 become col == 0

    count_opponent_main = np.count_nonzero(diagonal_main == (1 - game.current_player_idx))
    count_opponent_secondary = np.count_nonzero(diagonal_secondary == (1 -  game.current_player_idx))

    if (diagonal_main[0] == -1 or diagonal_main[4] == -1) and count_opponent_main >= 4:
      count_row_0 = np.count_nonzero(board[0,:] != (1 - game.current_player_idx))   # it counts the number of cubes different to the opponent's ones on the row 0
      count_column_0 = np.count_nonzero(board[:,0] != (1 - game.current_player_idx))    # it counts the number of cubes different to the opponent's ones on the column 0
      if diagonal_main[0] == -1 and (count_row_0 > 0 or count_column_0 > 0):   # we have to guarantee that on that row or column there is at least a cube that isn't opponent's one
        if count_row_0 > 1 or (count_row_0 == 1 and board[0,4] == (1-game.current_player_idx)): # we have to guarantee that free cube or agent's cube isn't on the other extreme (because we can't put it in the same position)
            return True
        if count_column_0 > 1 or (count_column_0 == 1 and board[4,0] == (1-game.current_player_idx)): # we have to guarantee that free cube or agent's cube isn't on the other extreme (because we can't put it in the same position)
            return True

      count_row_4 = np.count_nonzero(board[4,:] != (1 - game.current_player_idx))   # it counts the number of cubes different to the opponent's ones on the row 4
      count_column_4 = np.count_nonzero(board[:,4] != (1 - game.current_player_idx))    # it counts the number of cubes different to the opponent's ones on the column 4
      if diagonal_main[4] == -1 and (count_row_4 > 0 or count_column_4 > 0) and board[4,0] != game.current_player_idx:  # we have to guarantee that on that row or column there is at least a cube that isn't opponent's one and that it isn't on the other extreme (because we can't put it in the same position)
        if count_row_4 > 1 or (count_row_4 == 1 and board[4,0] == (1-game.current_player_idx)): # we have to guarantee that free cube or agent's cube isn't on the other extreme (because we can't put it in the same position)
            return True
        if count_column_4 > 1 or (count_column_4 == 1 and board[0,4] == (1-game.current_player_idx)): # we have to guarantee that free cube or agent's cube isn't on the other extreme (because we can't put it in the same position)
            return True

    if (diagonal_secondary[0] == -1 or diagonal_secondary[4] == -1) and count_opponent_secondary >= 4:
      count_row_0 = np.count_nonzero(board[0,:] != (1 - game.current_player_idx))   # it counts the number of cubes different to the opponent's ones on the row 0
      count_column_4 = np.count_nonzero(board[:,4] != (1 - game.current_player_idx))    # it counts the number of cubes different to the opponent's ones on the column 4
      if diagonal_secondary[0] == -1 and (count_row_0 > 0 or count_column_4 > 0) and board[0,0] != game.current_player_idx:   # we have to guarantee that on that row or column there is at least a cube that isn't opponent's one and that it isn't on the other extreme (because we can't put it in the same position)
        if count_row_0 > 1 or (count_row_0 == 1 and board[0,0] == (1-game.current_player_idx)): # we have to guarantee that free cube or agent's cube isn't on the other extreme (because we can't put it in the same position)
            return True
        if count_column_4 > 1 or (count_column_4 == 1 and board[4,4] == (1-game.current_player_idx)): # we have to guarantee that free cube or agent's cube isn't on the other extreme (because we can't put it in the same position)
            return True

      count_row_4 = np.count_nonzero(board[4,:] != (1 - game.current_player_idx))   # it counts the number of cubes different to the opponent's ones on the row 4
      count_column_0 = np.count_nonzero(board[:,0] != (1 - game.current_player_idx))    # it counts the number of cubes different to the opponent's ones on the column 0
      if diagonal_secondary[4] == -1 and (count_row_4 > 0 or count_column_0 > 0) and board[0,4] != game.current_player_idx:  # we have to guarantee that on that row or column there is at least a cube that isn't opponent's one and that it isn't on the other extreme (because we can't put it in the same position)
        if count_row_4 > 1 or (count_row_4 == 1 and board[4,4] == (1-game.current_player_idx)): # we have to guarantee that free cube or agent's cube isn't on the other extreme (because we can't put it in the same position)
            return True
        if count_column_0 > 1 or (count_column_0 == 1 and board[0,0] == (1-game.current_player_idx)): # we have to guarantee that free cube or agent's cube isn't on the other extreme (because we can't put it in the same position)
            return True

    return False


def r9(game: 'MyGame'):

    '''
      Blocks the opponent's win: if the opponent is going to make 5 on the two main diagonals,  block the winning move (if and only if
      at the corners there are cubes with our face or without face)
    '''


    board = game.get_board()

    diagonal_main = np.diagonal(board)
    diagonal_secondary = np.diagonal(board[:, ::-1]) # [:, ::-1] returns the matrix with inverted columns col == 4 become col == 0

    count_opponent_main = np.count_nonzero(diagonal_main == (1 - game.current_player_idx))

    count_opponent_secondary = np.count_nonzero(diagonal_secondary == (1 -  game.current_player_idx))

    if (diagonal_main[0] == -1 or diagonal_main[4] == -1) and count_opponent_main >= 4:
      if diagonal_main[0] == -1 and (board[0,1] == game.current_player_idx or board[1,0] == game.current_player_idx):
        return True
      if diagonal_main[4] == -1 and (board[3,4] == game.current_player_idx or board[4,3] == game.current_player_idx):
        return True

    if (diagonal_secondary[0] == -1 or diagonal_secondary[4] == -1) and count_opponent_secondary >= 4:
      if diagonal_secondary[0] == -1 and (board[0,3] == game.current_player_idx or board[1,4] == game.current_player_idx):
        return True
      if diagonal_secondary[4] == -1 and (board[3,0] == game.current_player_idx or board[4,1] == game.current_player_idx):
        return True

    return False


def r10(game: 'MyGame'):
    '''
      If on one row there are at least 3 cubes with our symbol, then we try to put a cube there, even if it is an inner row
    '''
    board = game.get_board()

    for row_index,row in enumerate(board):
        if len(row[row ==  game.current_player_idx]) >= 3 and len(row[row == -1]) > 0:
            if row_index > 0 and row_index < 4:
                if not (row[0]== (1 -  game.current_player_idx) and row[4]== (1-  game.current_player_idx)):  # not both extremes are opponent's cubes
                    if not ((row[0]== game.current_player_idx and row[4]== (1-  game.current_player_idx)) or (row[0]== (1- game.current_player_idx) and row[4]==  game.current_player_idx) or (row[0]== game.current_player_idx and row[4]== game.current_player_idx)): # at least one extreme must be without face
                        return True
            else:
                return True
    return False

def r11(game: 'MyGame'):

    '''
      if on one column there are at least 3 cubes with our symbols, then we try to put a cube there, even if it is an inner column
    '''
    board = game.get_board()

    for col_index, col in enumerate(board.T):
        if len(col[col == game.current_player_idx]) >= 3 and len(col[col == -1]) > 0:
            if col_index > 0 and col_index < 4:
                if not (col[0]== (1 - game.current_player_idx) and col[4]== (1- game.current_player_idx)):  # not both extremes are opponent's cubes
                    if not ((col[0]== game.current_player_idx and col[4]== (1- game.current_player_idx)) or (col[0]== (1- game.current_player_idx) and col[4]== game.current_player_idx) or (col[0]== game.current_player_idx and col[4]== game.current_player_idx)): # at least one cube must be without face
                        return True
            else:
                return True
    return False

def r12(game: "MyGame"):

    '''
      It blocks the opponent's win: if the opponent has at least 3 cubes with its symbol on the row, the agent blocks the winning move pushing its cube
    '''

    board = game.get_board()

    for row_index, row in enumerate(board):
        if len(row[row==(1- game.current_player_idx)]) >= 3 and len(row[row==-1]) > 0:
            if row_index > 0 and row_index < 4:
                if not (row[0]== (1- game.current_player_idx) and row[4]== (1- game.current_player_idx)):
                    if not ((row[0]== game.current_player_idx and row[4]== (1- game.current_player_idx)) or (row[0]== (1- game.current_player_idx) and row[4]== game.current_player_idx)):
                        return True
            else:
                return True

    return False

def r13(game: "MyGame"):

    '''
      It blocks the opponent's win: if the opponent has at least 3 cubes with its symbol on the column, the agent blocks the winning move pushing its cube
    '''

    board = game.get_board()

    for col_index, col in enumerate(board.T):
        if len(col[col== (1-game.current_player_idx)]) >= 3 and len(col[col==-1]) > 0:
            if col_index > 0 and col_index < 4:
                if not (col[0]== (1- game.current_player_idx) and col[4]== (1- game.current_player_idx)):
                    if not ((col[0]== game.current_player_idx and col[4]== (1-game.current_player_idx)) or (col[0]==(1- game.current_player_idx) and col[4]== game.current_player_idx)):
                        return True
            else:
                return True

    return False

def r14(game: "MyGame"):

    '''
      if it find an available cube or with player's face on the perimeter,  do a random move from those allowed.
    '''

    board = game.get_board()

    for row_index, row in enumerate(board):
        if row_index == 0 or row_index == 4:
            for el in row:
                if el != (1- game.current_player_idx):
                    return True

        else: # internal rows
            if row[0] != (1- game.current_player_idx) or row[4] != (1- game.current_player_idx):
                return True



    return False



# Rules

In [ ]:
CONDITIONS = [
    r1,
    r2,
    r3,
    r4,
    r5,
    r6,
    r7,
    r8,
    r9,
    r10,
    r11,
    r12,
    r13,
    r14
]

# Actions

In [ ]:
def action1(game: 'MyGame'):
    # Put your cube face in the row without any cube that shows your face
    r = -1
    col = -1
    move = -1
    board_situation = game.get_board()
    for row_index, row in enumerate(board_situation):
        result = np.all(row != game.current_player_idx)
        if result == True:
            if row_index >= 1 and row_index <= 3: # Internal rows
                if row[0] == -1 and row[4] == -1:
                    col = random.choice([0, 4])
                    if col == 0:
                        move = Move.RIGHT
                    else: # col ==4:
                        move = Move.LEFT
                    r = row_index
                    break
                elif row[0] == -1:
                    col = 0
                    move = Move.RIGHT
                    r = row_index
                    break
                elif row[4] == -1 :
                    col = 4
                    move = Move.LEFT
                    r = row_index
                    break
            else:  # first row or last row
                # takes a column != 1
                free = [index for index,elem in enumerate(row) if elem==-1] # list of column indexes with no face cubes
                available = [index for index,elem in enumerate(row) if elem==game.current_player_idx] # list of column indexes with agent's cubes
                if len(free) > 0:
                  c = random.choice(free)
                  if row_index == 0:  # first
                    r = row_index
                    if c == 0:
                      return ((c,r), Move.RIGHT)
                    elif c==4:
                      return ((c,r), Move.LEFT)
                    else:
                      return ((c,r), random.choice([Move.LEFT, Move.RIGHT]))
                  else:  # last
                    r = row_index
                    if c == 0:
                      return ((c,r), Move.RIGHT)
                    elif c==4:
                      return ((c,r), Move.LEFT)
                    else:
                      return ((c,r), random.choice([Move.LEFT, Move.RIGHT]))
                elif len(available) > 0:
                  c = random.choice(available)
                  if row_index == 0:  # first
                    r = row_index
                    if c == 0:
                      return ((c,r), Move.RIGHT)
                    elif c==4:
                      return ((c,r), Move.LEFT)
                    else:
                      return ((c,r), random.choice([Move.LEFT, Move.RIGHT]))
                  else:  # last
                    r = row_index
                    if c == 0:
                      return ((c,r), Move.RIGHT)
                    elif c==4:
                      return ((c,r), Move.LEFT)
                    else:
                      return ((c,r), random.choice([Move.LEFT, Move.RIGHT]))

    #returm from_pos, move
    return ((col , r), move)

def action2(game: 'MyGame'):
    # put the your cube face in the cube without any cube that shows your face

    r = -1
    c = -1
    move = -1

    board_situation = game.get_board()
    board_situation = board_situation.T
    for col_index, col in enumerate(board_situation):
        result = np.all(col != game.current_player_idx)
        if result == True:
            if col_index >= 1 and col_index <= 3:  # internal column
                if col[0] == -1 and col[4] == -1:
                    r = random.choice([0, 4])
                    if r == 0:
                        move = Move.BOTTOM
                    else:
                        move = Move.TOP
                    c = col_index
                    break
                elif col[0] == -1:
                    r = 0
                    move = Move.BOTTOM
                    c = col_index
                    break
                elif col[4] == -1:
                    r = 4
                    move = Move.TOP
                    c = col_index
                    break
            else:
                # takes a row != 1
                free = [index for index,elem in enumerate(col) if elem==-1]
                available = [index for index,elem in enumerate(col) if elem==game.current_player_idx]
                if len(free) > 0:
                  r = random.choice(free)
                  if col_index == 0:  # first
                    c = col_index
                    if r == 0:
                      return ((c,r), Move.BOTTOM)
                    elif r==4:
                      return ((c,r), Move.TOP)
                    else:
                      return ((c,r), random.choice([Move.BOTTOM, Move.TOP]))
                  else:  # last
                    c = col_index
                    if r == 0:
                      return ((c,r), Move.BOTTOM)
                    elif r==4:
                      return ((c,r), Move.TOP)
                    else:
                      return ((c,r), random.choice([Move.BOTTOM, Move.TOP]))
                elif len(available) > 0:
                  r = random.choice(free)
                  if col_index == 0:  # first
                    c = col_index
                    if r == 0:
                      return ((c,r), Move.BOTTOM)
                    elif r==4:
                      return ((c,r), Move.TOP)
                    else:
                      return ((c,r), random.choice([Move.BOTTOM, Move.TOP]))
                  else:  # last
                    c = col_index
                    if r == 0:
                      return ((c,r), Move.BOTTOM)
                    elif r==4:
                      return ((c,r), Move.TOP)
                    else:
                      return ((c,r), random.choice([Move.BOTTOM, Move.TOP]))

    #returm from_pos, move
    return ((c , r), move)

def action3(game: 'MyGame'):
    '''
      If we are doing 5 on one row, on the perimeter
      of the board add cube with our face
    '''
    r = -1
    col = -1
    move = -1

    board_situation = game.get_board()
    for row_index, row in enumerate(board_situation):
        count_agent = 0
        for el in row:
            if el == game.current_player_idx:
                count_agent += 1
                if count_agent >= 4 and (row[0] == -1 or row[4] == -1): # There are four consecutive 0 in a row and there is the row[0] or row[4] available ( == -1)
                    r = row_index
                    if(row[0] == -1):
                        col = 0
                        move = Move.RIGHT
                        break
                    else:
                        col = 4
                        move = Move.LEFT
                        break
            else:
                count_agent = 0


    #returm from_pos, move
    return ((col , r), move)


def action4(game: 'MyGame'):
    '''
      If we are doing 5 on one coumn, on the perimeter
      of the board add cube with our face
    '''
    row = -1
    c = -1
    move = -1

    board_situation = game.get_board()
    for col_index, col in enumerate(board_situation.T):
        count_agent = 0
        for el in col:
            if el == game.current_player_idx:
                count_agent += 1
                if count_agent >= 4 and (col[0] == -1 or col[4] == -1): # There are four consecutive 0 in a col and there is the col[0] or col[4] available ( == -1)
                    c = col_index
                    if(col[0] == -1):
                        row = 0
                        move = Move.BOTTOM
                        break
                    else:
                        row = 4
                        move = Move.TOP
                        break
            else:
                count_agent = 0


    #returm from_pos, move
    return ((c , row), move)


def action5(game: 'MyGame'):
    row = -1
    col = -1
    move = -1

    '''
      If we are doing 5 on one of the two main diagonals , on the perimeter
      of the board add cube with our sign on the corner which complete the 5-sequence (if and only if that corner is available)
    '''

    board_situation = game.get_board()

    diagonal_main = np.diagonal(board_situation)
    diagonal_secondary = np.diagonal(board_situation[:, ::-1]) # [:, ::-1] returns the matrix with inverted columns col == 4 become col == 0

    count_non_agent_main = np.count_nonzero(diagonal_main != game.current_player_idx)

    count_non_agent_secondary = np.count_nonzero(diagonal_secondary != game.current_player_idx)

    if (diagonal_main[0] == -1 or diagonal_main[4] == -1) and count_non_agent_main == 1:
        if diagonal_main[0] == -1:
            if board_situation[0, 1] == game.current_player_idx:  #board_situation[row, column]
                col = 0
                row = 0
                move = Move.RIGHT
                return ((col, row), move)

            if board_situation[1, 0] == game.current_player_idx:
                col = 0
                row = 0
                move = Move.BOTTOM
                return ((col, row), move)

        # I don't put else here cuz of the return at the previous line
        if diagonal_main[4] == -1:
            if board_situation[4, 3] == game.current_player_idx:
                col = 4
                row = 4
                move = Move.LEFT
                return ((col, row), move)

            if board_situation[3, 4] == game.current_player_idx:
                col = 4
                row = 4
                move = Move.TOP
                return ((col, row), move)


    if (diagonal_secondary[0] == -1 or diagonal_secondary[4] == -1 and count_non_agent_secondary == 1):
        if diagonal_secondary[0] == -1:
            if board_situation[0, 3] == game.current_player_idx: # board_situation[row, column]
                col = 4
                row = 0
                move = Move.LEFT
                return ((col, row), move)

            if board_situation[1, 4] == game.current_player_idx:
                col = 4
                row = 0
                move = Move.BOTTOM
                return ((col, row), move)


        # I don't put else here cuz of the return at the previous line
        if diagonal_secondary[4] == -1:
            if board_situation[3, 0] == game.current_player_idx:
                col = 0
                row = 4
                move = Move.TOP
                return ((col, row), move)

            if board_situation[4, 1] == game.current_player_idx:
                col = 0
                row = 4
                move = Move.RIGHT
                return ((col, row), move)



    return ((0,0), Move.TOP)  # Since the action is exectued cuz the rule 5 is active this return will never exececuted
                              # We need to put this statement here just for sintax


def action6(game: 'MyGame'):

    '''
      blocks the opponent's win: if the opponent is going to make 5 on the row, block the winning move (if and only if
      at the extreme there are cubes with our face or without face)

    '''
    r = -1
    col = -1
    move = -1
    board_situation = game.get_board()
    for row_index, row in enumerate(board_situation):
        count_opponent = 0
        for el in row:
            if el == (1- game.current_player_idx):
                count_opponent += 1
        if count_opponent >= 4 and (row[0] == -1 or row[4] == -1): # There are four consecutive 1 in a row and there is the row[0] or row[4] available ( == -1)
            r = row_index
            if(row[0] == -1):
                col = 0
                move = Move.RIGHT
                break
            else:
                col = 4
                move = Move.LEFT
                break

    return ((col , r), move)


def action7(game: 'MyGame'):

    '''
        blocks the opponent's win: if the opponent is going to make 5 on the column, block the winning move (if and only if
        at the extreme there are cubes with our face or without face)

    '''
    row = -1
    c = -1
    move = -1

    board_situation = game.get_board()
    for col_index, col in enumerate(board_situation.T):
        count_opponent = 0
        for el in col:
            if el ==  (1- game.current_player_idx):
                count_opponent += 1
        if count_opponent >= 4 and (col[0] == -1 or col[4] == -1): # There are four consecutive 1 in a column and there is the column[0] or column[4] available ( == -1)
            c = col_index
            if(col[0] == -1):
                row = 0
                move = Move.BOTTOM
                break
            else:
                row = 4
                move = Move.TOP
                break

    #returm from_pos, move
    return ((c , row), move)


def action8(game: 'MyGame'):
    # Do a bad move ---> this is import to see if the associated rules goes down (at the end the associated rules must have a low weight)
    r = -1
    c = -1

    # row
    board_situation = game.get_board()

    for row_index, row in enumerate(board_situation):
        count_opponent = 0
        count_free = 0
        for el in row:
            if el == (1-game.current_player_idx):
                count_opponent += 1
            if el == -1:
                count_free += 1

        if count_opponent == 4 and count_free == 1: # There are four opponent's cubes in a row and there is one available ( == -1)
            r = row_index   # row where opponent is winning
            possible_rows = [ i for i in range(BOARD_SIZE) if i != r]   # list of all row indexes, with the exception of the one where the opponent is winning
            for i in possible_rows: # I search in the rows where the opponent is not winning in order to do a bad moves and allow the opponent to win
                if i == 0 or i == 4: # if we are looking at the first or last row
                    for col_index, el_2 in enumerate(range(BOARD_SIZE)):
                        if el_2 != (1- game.current_player_idx):
                            if col_index == 0:
                                return ((col_index, i), Move.RIGHT)
                            elif col_index == 4:
                                return ((col_index, i), Move.LEFT)
                            else:   # inner columns
                                return ((col_index, i), random.choice([Move.RIGHT, Move.LEFT])) # not TOP and BOTTOM in order to not modify opponent's winning row
                else: # if we are looking at the intermediate rows, so we have to check for a 0 or -1 in col == 0 or in col == 4
                    if board_situation[i][0] != (1- game.current_player_idx):
                        return ((0, i), Move.RIGHT)
                    if board_situation[i][4] != (1- game.current_player_idx):
                            return ((4, i), Move.LEFT)

    # column
    board_situation = game.get_board().T    # transposed

    for col_index, col in enumerate(board_situation):
        count_opponent = 0
        count_free = 0
        for el in col:
            if el == (1-game.current_player_idx):
                count_opponent += 1
            if el == -1:
                count_free += 1

        if count_opponent == 4 and count_free == 1: # There are four opponent's cubes in a column and there is one available ( == -1)
            c = col_index   # column where opponent is winning
            possible_cols = [ i for i in range(BOARD_SIZE) if i != c]   # list of all col indexes, with the exception of the one where the opponent is winning
            for i in possible_cols: # I search in the rows where the opponent is not winning in order to do a bad moves and allow the opponent to win
                if i == 0 or i == 4: # if we are looking at the first or last column
                    for row_index, el_2 in enumerate(range(BOARD_SIZE)):
                        if el_2 != (1- game.current_player_idx):
                            if row_index == 0:
                                return ((i, row_index), Move.BOTTOM)
                            elif row_index == 4:
                                return ((i, row_index), Move.TOP)
                            else:   # inner rows
                                return ((i, row_index), random.choice([Move.TOP, Move.BOTTOM])) # not LEFT and RIGHT in order to not modify opponent's winning column
                else: # if we are looking at the intermediate columns, so we have to check for a 0 or -1 in row == 0 or in row == 4
                    if board_situation[i][0] != (1- game.current_player_idx):
                        return ((i, 0), Move.BOTTOM)
                    if board_situation[i][4] != (1- game.current_player_idx):
                        return ((i, 4), Move.TOP)

    # diagonal
    board_situation = game.get_board()

    diagonal_main = np.diagonal(board_situation)
    diagonal_secondary = np.diagonal(board_situation[:, ::-1]) # [:, ::-1] returns the matrix with inverted columns col == 4 become col == 0

    count_opponent_main = np.count_nonzero(diagonal_main == (1- game.current_player_idx))
    count_opponent_secondary = np.count_nonzero(diagonal_secondary == (1- game.current_player_idx))

    count_free_main = np.count_nonzero(diagonal_main == -1)
    count_free_secondary = np.count_nonzero(diagonal_secondary == -1)

    if count_opponent_main == 4 and count_free_main == 1:
        #idx = diagonal_main.index(-1)
        idx = np.where(diagonal_main == -1)[0]
        if idx == 0:  # coordinate where the opponent could win
            r = 0
            c = 0

            # row
            possible_cols = [ j for j, el in enumerate(board_situation[r]) if el != (1-game.current_player_idx) and j != c]    # we are searching cubes on that row that has not opponent's face
            if len(possible_cols) > 0:
                if len(possible_cols) == 1 and board_situation[0,4] == (1 - game.current_player_idx):
                    col = possible_cols[0]
                    return ((col, r), Move.RIGHT)
                elif len(possible_cols) > 1:
                    if BOARD_SIZE-1 in possible_cols:
                        possible_cols.remove(BOARD_SIZE-1)
                    if len(possible_cols) == 1:
                        col = possible_cols[0]
                    else:
                        col = choice(possible_cols)
                    return ((col, r), Move.RIGHT)

            # column
            possible_rows = [ j for j, el in enumerate(board_situation[:, c]) if el != (1-game.current_player_idx) and j != r]    # we are searching cubes on that column that has not opponent's face
            if len(possible_rows) > 0:
                if len(possible_rows) == 1 and board_situation[4,0] == (1 - game.current_player_idx):
                    row = possible_rows[0]
                    return ((c, row), Move.BOTTOM)
                elif len(possible_rows) > 1:
                    if BOARD_SIZE-1 in possible_rows:
                        possible_rows.remove(BOARD_SIZE-1)
                    if len(possible_rows) == 1:
                        row = possible_rows[0]
                    else:
                        row = choice(possible_rows)
                    return ((c, row), Move.BOTTOM)

        if idx == 4:
            r = 4
            c = 4

            # row
            possible_cols = [ j for j, el in enumerate(board_situation[r]) if el != (1-game.current_player_idx) and j != c]    # we are searching cubes on that row that has not opponent's face
            if len(possible_cols) > 0:
                if len(possible_cols) == 1 and board_situation[0,4] == (1 - game.current_player_idx):
                    col = possible_cols[0]
                    return ((col, r), Move.LEFT)
                elif len(possible_cols) > 1:
                    if 0 in possible_cols:
                        possible_cols.remove(0)
                    if len(possible_cols) == 1:
                        col = possible_cols[0]
                    else:
                        col = choice(possible_cols)
                    return ((col, r), Move.LEFT)

            # column
            possible_rows = [ j for j, el in enumerate(board_situation[:, c]) if el != (1-game.current_player_idx) and j != r]    # we are searching cubes on that column that has not opponent's face
            if len(possible_rows) > 0:
                if len(possible_rows) == 1 and board_situation[4,0] == (1 - game.current_player_idx):
                    row = possible_rows[0]
                    return ((c, row), Move.TOP)
                elif len(possible_rows) > 1:
                    if 0 in possible_rows:
                        possible_rows.remove(0)
                    if len(possible_rows) == 1:
                        row = possible_rows[0]
                    else:
                        row = choice(possible_rows)
                    return ((c, row), Move.TOP)

    if count_opponent_secondary == 4 and count_free_secondary == 1:
        #idx = diagonal_main.index(-1)
        idx = np.where(diagonal_secondary == -1)[0]
        if idx==0:
            r = 0
            c = 4

            # row
            possible_cols = [ j for j, el in enumerate(board_situation[r]) if el != (1-game.current_player_idx) and j != c]    # we are searching cubes on that row that has not opponent's face
            if len(possible_cols) > 0:
                if len(possible_cols) == 1 and board_situation[0,0] == (1 - game.current_player_idx):
                    col = possible_cols[0]
                    return ((col, r), Move.LEFT)
                elif len(possible_cols) > 1:
                    if 0 in possible_cols:
                        possible_cols.remove(0)
                    if len(possible_cols) == 1:
                        col = possible_cols[0]
                    else:
                        col = choice(possible_cols)
                    return ((col, r), Move.LEFT)

            # column
            possible_rows = [ j for j, el in enumerate(board_situation[:, c]) if el != (1-game.current_player_idx) and j != r]    # we are searching cubes on that column that has not opponent's face
            if len(possible_rows) > 0:
                if len(possible_rows) == 1 and board_situation[4,4] == (1 - game.current_player_idx):
                    row = possible_rows[0]
                    return ((c, row), Move.BOTTOM)
                elif len(possible_rows) > 1:
                    if BOARD_SIZE-1 in possible_rows:
                        possible_rows.remove(BOARD_SIZE-1)
                    if len(possible_rows) == 1:
                        row = possible_rows[0]
                    else:
                        row = choice(possible_rows)
                    return ((c, row), Move.BOTTOM)

        if idx==4:
            r = 4
            c = 0

            # row
            possible_cols = [ j for j, el in enumerate(board_situation[r]) if el != (1-game.current_player_idx) and j != c]    # we are searching cubes on that row that has not opponent's face
            if len(possible_cols) > 0:
                if len(possible_cols) == 1 and board_situation[4,4] == (1 - game.current_player_idx):
                    col = possible_cols[0]
                    return ((col, r), Move.RIGHT)
                elif len(possible_cols) > 1:
                    if BOARD_SIZE-1 in possible_cols:
                        possible_cols.remove(BOARD_SIZE-1)
                    if len(possible_cols) == 1:
                        col = possible_cols[0]
                    else:
                        col = choice(possible_cols)
                    return ((col, r), Move.RIGHT)

            # column
            possible_rows = [ j for j, el in enumerate(board_situation[:, c]) if el != (1-game.current_player_idx) and j != r]    # we are searching cubes on that column that has not opponent's face
            if len(possible_rows) > 0:
                if len(possible_rows) == 1 and board_situation[0,0] == (1 - game.current_player_idx):
                    row = possible_rows[0]
                    return ((c, row), Move.TOP)
                elif len(possible_rows) > 1:
                    if 0 in possible_rows:
                        possible_rows.remove(0)
                    if len(possible_rows) == 1:
                        row = possible_rows[0]
                    else:
                        row = choice(possible_rows)
                    return ((c, row), Move.TOP)

    return ((-1 , -1), -1) # This return will never exececuted
                              # We need to put this statement here just for sintax



def action9(game: 'MyGame'):

    '''
      Blocks the opponent's win: if the opponent is going to make 5 on the two main diagonals,  block the winning move (if and only if
      at the corners there are cubes with our face or without face)
    '''
    row = -1
    col = -1
    move = -1
    board_situation = game.get_board()


    diagonal_main = np.diagonal(board_situation)
    diagonal_secondary = np.diagonal(board_situation[:, ::-1]) # [:, ::-1] returns the matrix with inverted columns col == 4 become col == 0

    count_opponent_main = np.count_nonzero(diagonal_main == (1- game.current_player_idx))

    count_opponent_secondary = np.count_nonzero(diagonal_secondary == (1- game.current_player_idx))


    if (diagonal_main[0] == -1 or diagonal_main[4] == -1) and count_opponent_main >= 4:
        if diagonal_main[0] == -1:
            if board_situation[0, 1] == game.current_player_idx: #board_situation[row, column]
                col = 0
                row = 0
                move = Move.RIGHT
                return ((col, row), move)

            if board_situation[1, 0] == game.current_player_idx:
                col = 0
                row = 0
                move = Move.BOTTOM
                return ((col, row), move)


        # I don't put else here cuz of the return at the previous line
        if diagonal_main[4] == -1:
            if board_situation[4, 3] == game.current_player_idx:
                col = 4
                row = 4
                move = Move.LEFT
                return ((col, row), move)

            if board_situation[3, 4] == game.current_player_idx:
                col = 4
                row = 4
                move = Move.TOP
                return ((col, row), move)



    if (diagonal_secondary[0] == -1 or diagonal_secondary[4] == -1 and count_opponent_secondary >= 4):
        if diagonal_secondary[0] == -1:
            if board_situation[0, 3] == game.current_player_idx:
                col = 4
                row = 0
                move = Move.LEFT
                return ((col, row), move)

            if board_situation[1, 4] == game.current_player_idx:
                col = 4
                row = 0
                move = Move.BOTTOM
                return ((col, row), move)


        # I don't put else here cuz of the return at the previous line
        if diagonal_secondary[4] == -1:
            if board_situation[3, 0] == game.current_player_idx:
                col = 0
                row = 4
                move = Move.TOP
                return ((col, row), move)

            if board_situation[4, 1] == game.current_player_idx:
                col = 0
                row = 4
                move = Move.RIGHT
                return ((col, row), move)


    return ((0,0), Move.TOP)  # Since the action is exectued cuz the rule 9 is active this return will never exececuted
                              # We need to put this statement here just for sintax

def action10(game: 'MyGame'):
    # If on one row there are at least 3 cubes with our symbol, then we try to put a cube there
    r = -1
    c = -1
    move = -1
    board_situation = game.get_board()
    for row_index, row in enumerate(board_situation):
        if len(row[row== game.current_player_idx]) >= 3 and len(row[row==-1]) > 0:
            if row_index >= 1 and row_index <= 3:   # inner rows
                r = row_index
                if(row[0] == -1):
                    c = 0
                    move = Move.RIGHT
                    break
                elif(row[4] == -1):
                    c = 4
                    move = Move.LEFT
                    break
            else:   # first or last row
                free = [index for index,elem in enumerate(row) if elem==-1] # list of indexes of cubes without face

                r = row_index
                c = random.choice(free)
                if row_index == 0:  # first row
                  if c == 0:
                    return ((c,r), Move.RIGHT)
                  elif c==4:
                    return ((c,r), Move.LEFT)
                  else:
                    return ((c,r), random.choice([Move.LEFT, Move.RIGHT]))
                else:  # last row
                  if c == 0:
                    return ((c,r), Move.RIGHT)
                  elif c==4:
                    return ((c,r), Move.LEFT)
                  else:
                    return ((c,r), random.choice([Move.LEFT, Move.RIGHT]))

    #return from_pos, move (in case of break)
    return ((c, r), move)

def action11(game: 'MyGame'):
    # If on one col there are at least 3 cubes with our symbol, then we try to put a cube there
    r = -1
    c = -1
    move = -1
    board_situation = game.get_board()
    for col_index, col in enumerate(board_situation.T):
        if len(col[col== game.current_player_idx]) >= 3 and len(col[col==-1]) > 0:
            if col_index >= 1 and col_index <= 3:   # inner columns
                c = col_index
                if(col[0] == -1):
                    r = 0
                    move = Move.BOTTOM
                    break
                elif(col[4] == -1):
                    r = 4
                    move = Move.TOP
                    break
            else:   # first or last column
                free = [index for index,elem in enumerate(col) if elem==-1] # list of indexes of cubes without face

                r = random.choice(free)
                c = col_index
                if col_index == 0:  # first
                  if r == 0:
                    return ((c,r), Move.BOTTOM)
                  elif r==4:
                    return ((c,r), Move.TOP)
                  else:
                    return ((c,r), random.choice([Move.BOTTOM, Move.TOP]))
                else:  # last
                  if r == 0:
                    return ((c,r), Move.BOTTOM)
                  elif r==4:
                    return ((c,r), Move.TOP)
                  else:
                    return ((c,r), random.choice([Move.BOTTOM, Move.TOP]))

    #return from_pos, move (in case of break)
    return ((c, r), move)

def action12(game: 'MyGame'):
    # If on one row there are at least 3 cubes with opponent's symbol, then we try to avoid his win by pushing a cube with our symbol
    r = -1
    c = -1
    move = -1
    board_situation = game.get_board()
    for row_index, row in enumerate(board_situation):
        if len(row[row== (1- game.current_player_idx)]) >= 3 and len(row[row==-1]) > 0:
            if row_index >= 1 and row_index <= 3:   # inner rows
                r = row_index
                if(row[0] == -1):
                    c = 0
                    move = Move.RIGHT
                    break
                elif(row[4] == -1):
                    c = 4
                    move = Move.LEFT
                    break
            else:   # first or last row
                free = [index for index,elem in enumerate(row) if elem==-1] # list of indexes of cubes without face

                r = row_index
                c = random.choice(free)
                if row_index == 0:  # first
                  if c == 0:
                    return ((c,r), Move.RIGHT)
                  elif c==4:
                    return ((c,r), Move.LEFT)
                  else:
                    return ((c,r), random.choice([Move.LEFT, Move.RIGHT]))
                else:  # last
                  if c == 0:
                    return ((c,r), Move.RIGHT)
                  elif c==4:
                    return ((c,r), Move.LEFT)
                  else:
                    return ((c,r), random.choice([Move.LEFT, Move.RIGHT]))

    #return from_pos, move (in case of break)
    return ((c, r), move)


def action13(game: "MyGame"):
    # If on one column there are at least 3 cubes with opponent's symbol, then we try to avoid his win by pushing a cube with our symbol
    r = -1
    c = -1
    move = -1

    board_situation = game.get_board()
    for col_index, col in enumerate(board_situation.T):
        if len(col[col== (1- game.current_player_idx)]) >= 3 and len(col[col==-1]) > 0:
            if col_index >= 1 and col_index <= 3:   # inner columns
                c = col_index
                if(col[0] == -1):
                    r = 0
                    move = Move.BOTTOM
                    break
                elif(col[4] == -1):
                    r = 4
                    move = Move.TOP
                    break
            else:   # first or last column
                free = [index for index,elem in enumerate(col) if elem==-1]

                c = col_index
                r = random.choice(free)
                if col_index == 0:  # first
                  if r == 0:
                    return ((c,r), Move.BOTTOM)
                  elif r==4:
                    return ((c,r), Move.TOP)
                  else:
                    return ((c,r), random.choice([Move.TOP, Move.BOTTOM]))
                else:  # last
                  if r == 0:
                    return ((c,r), Move.BOTTOM)
                  elif r==4:
                    return ((c,r), Move.TOP)
                  else:
                    return ((c,r), random.choice([Move.TOP, Move.BOTTOM]))

    #return from_pos, move (in case of break)
    return ((c, r), move)


def action14(game: "MyGame"):

    '''
      if it find an available cube or with player's face on the perimeter,  do a random move from those allowed.

    '''

    r = -1
    c = -1
    move = -1
    board = game.get_board()


    for row_index, row in enumerate(board):
        if row_index == 0 or row_index == 4: #external rows
            for col_index, el in enumerate(row):
                if el != (1- game.current_player_idx):
                    r = row_index
                    c = col_index
                    move = random.choice([Move.RIGHT, Move.LEFT])
                    break

        else: # internal rows
            if row[0] != (1- game.current_player_idx) or row[4] != (1- game.current_player_idx):
                r = row_index
                if row[0] != (1- game.current_player_idx):
                    c = 0
                    move = random.choice([Move.RIGHT, Move.TOP, Move.BOTTOM])
                    break
                if row[4] != (1- game.current_player_idx):
                    c = 4
                    move = random.choice([Move.LEFT, Move.TOP, Move.BOTTOM])
                    break

    return ((c, r), move)




In [ ]:
ACTIONS = [
    (action1, 0),
    (action2, 0),
    (action3, 0),
    (action4, 0),
    (action5, 0),
    (action6, 0),
    (action7, 0),
    (action8, 0),
    (action9, 0),
    (action10, 0),
    (action11, 0),
    (action12, 0),
    (action13, 0),
    (action14, 0)
]

## Game class definition
It contains *Move* class, *Player* class as interface for our players and ***Game*** class implemented by Andrea Calabrese.

In [ ]:
class Move(Enum):

    """
    Selects where you want to place the taken piece. The rest of the pieces are shifted
    """

    TOP = 0
    BOTTOM = 1
    LEFT = 2
    RIGHT = 3


class Player(ABC):
    def __init__(self) -> None:
        """You can change this for your player if you need to handle state/have memory"""
        pass

    @abstractmethod
    def make_move(self, game: "Game") -> tuple[tuple[int, int], Move]:
        """
        The game accepts coordinates of the type (X, Y). X goes from left to right, while Y goes from top to bottom, as in 2D graphics.
        Thus, the coordinates that this method returns shall be in the (X, Y) format.

        game: the Quixo game. You can use it to override the current game with yours, but everything is evaluated by the main game
        return values: this method shall return a tuple of X,Y positions and a move among TOP, BOTTOM, LEFT and RIGHT
        """
        pass

class Game(object):
    def __init__(self) -> None:
        self._board = np.ones((5, 5), dtype=np.uint8) * -1
        self.current_player_idx = 1

    def get_board(self) -> np.ndarray:
        '''
        Returns the board
        '''
        return deepcopy(self._board)

    def get_current_player(self) -> int:
        '''
        Returns the current player
        '''
        return deepcopy(self.current_player_idx)

    def print(self):
        '''Prints the board. -1 are neutral pieces, 0 are pieces of player 0, 1 pieces of player 1'''
        print(self._board)

    def check_winner(self) -> int:
        '''Check the winner. Returns the player ID of the winner if any, otherwise returns -1'''
        # for each row
        for x in range(self._board.shape[0]):
            # if a player has completed an entire row
            if self._board[x, 0] != -1 and all(self._board[x, :] == self._board[x, 0]):
                # return the relative id
                return self._board[x, 0]
        # for each column
        for y in range(self._board.shape[1]):
            # if a player has completed an entire column
            if self._board[0, y] != -1 and all(self._board[:, y] == self._board[0, y]):
                # return the relative id
                return self._board[0, y]
        # if a player has completed the principal diagonal
        if self._board[0, 0] != -1 and all(
            [self._board[x, x]
                for x in range(self._board.shape[0])] == self._board[0, 0]
        ):
            # return the relative id
            return self._board[0, 0]
        # if a player has completed the secondary diagonal
        if self._board[0, -1] != -1 and all(
            [self._board[x, -(x + 1)]
             for x in range(self._board.shape[0])] == self._board[0, -1]
        ):
            # return the relative id
            return self._board[0, -1]
        return -1

    def play(self, player1: Player, player2: Player) -> int:
        '''Play the game. Returns the winning player'''
        players = [player1, player2]
        winner = -1
        while winner < 0:
            self.current_player_idx += 1
            self.current_player_idx %= len(players)
            ok = False
            while not ok:
                from_pos, slide = players[self.current_player_idx].make_move(
                    self)
                ok = self.__move(from_pos, slide, self.current_player_idx)

            winner = self.check_winner()
        return winner

    def __move(self, from_pos: tuple[int, int], slide: Move, player_id: int) -> bool:
        '''Perform a move'''
        if player_id > 2:
            return False
        # Oh God, Numpy arrays
        prev_value = deepcopy(self._board[(from_pos[1], from_pos[0])])
        acceptable = self.__take((from_pos[1], from_pos[0]), player_id)
        if acceptable:
            acceptable = self.__slide((from_pos[1], from_pos[0]), slide)
            if not acceptable:
                self._board[(from_pos[1], from_pos[0])] = deepcopy(prev_value)
        return acceptable

    def __take(self, from_pos: tuple[int, int], player_id: int) -> bool:
        '''Take piece'''
        # acceptable only if in border
        acceptable: bool = (
            # check if it is in the first row
            (from_pos[0] == 0 and from_pos[1] < 5)
            # check if it is in the last row
            or (from_pos[0] == 4 and from_pos[1] < 5)
            # check if it is in the first column
            or (from_pos[1] == 0 and from_pos[0] < 5)
            # check if it is in the last column
            or (from_pos[1] == 4 and from_pos[0] < 5)
            # and check if the piece can be moved by the current player
        ) and (self._board[from_pos] < 0 or self._board[from_pos] == player_id)
        if acceptable:
            self._board[from_pos] = player_id
        return acceptable

    def __slide(self, from_pos: tuple[int, int], slide: Move) -> bool:
        '''Slide the other pieces'''
        # define the corners
        SIDES = [(0, 0), (0, 4), (4, 0), (4, 4)]
        # if the piece position is not in a corner
        if from_pos not in SIDES:
            # if it is at the TOP, it can be moved down, left or right
            acceptable_top: bool = from_pos[0] == 0 and (
                slide == Move.BOTTOM or slide == Move.LEFT or slide == Move.RIGHT
            )
            # if it is at the BOTTOM, it can be moved up, left or right
            acceptable_bottom: bool = from_pos[0] == 4 and (
                slide == Move.TOP or slide == Move.LEFT or slide == Move.RIGHT
            )
            # if it is on the LEFT, it can be moved up, down or right
            acceptable_left: bool = from_pos[1] == 0 and (
                slide == Move.BOTTOM or slide == Move.TOP or slide == Move.RIGHT
            )
            # if it is on the RIGHT, it can be moved up, down or left
            acceptable_right: bool = from_pos[1] == 4 and (
                slide == Move.BOTTOM or slide == Move.TOP or slide == Move.LEFT
            )
        # if the piece position is in a corner
        else:
            # if it is in the upper left corner, it can be moved to the right and down
            acceptable_top: bool = from_pos == (0, 0) and (
                slide == Move.BOTTOM or slide == Move.RIGHT)
            # if it is in the lower left corner, it can be moved to the right and up
            acceptable_left: bool = from_pos == (4, 0) and (
                slide == Move.TOP or slide == Move.RIGHT)
            # if it is in the upper right corner, it can be moved to the left and down
            acceptable_right: bool = from_pos == (0, 4) and (
                slide == Move.BOTTOM or slide == Move.LEFT)
            # if it is in the lower right corner, it can be moved to the left and up
            acceptable_bottom: bool = from_pos == (4, 4) and (
                slide == Move.TOP or slide == Move.LEFT)
        # check if the move is acceptable
        acceptable: bool = acceptable_top or acceptable_bottom or acceptable_left or acceptable_right
        # if it is
        if acceptable:
            # take the piece
            piece = self._board[from_pos]
            # if the player wants to slide it to the left
            if slide == Move.LEFT:
                # for each column starting from the column of the piece and moving to the left
                for i in range(from_pos[1], 0, -1):
                    # copy the value contained in the same row and the previous column
                    self._board[(from_pos[0], i)] = self._board[(
                        from_pos[0], i - 1)]
                # move the piece to the left
                self._board[(from_pos[0], 0)] = piece
            # if the player wants to slide it to the right
            elif slide == Move.RIGHT:
                # for each column starting from the column of the piece and moving to the right
                for i in range(from_pos[1], self._board.shape[1] - 1, 1):
                    # copy the value contained in the same row and the following column
                    self._board[(from_pos[0], i)] = self._board[(
                        from_pos[0], i + 1)]
                # move the piece to the right
                self._board[(from_pos[0], self._board.shape[1] - 1)] = piece
            # if the player wants to slide it upward
            elif slide == Move.TOP:
                # for each row starting from the row of the piece and going upward
                for i in range(from_pos[0], 0, -1):
                    # copy the value contained in the same column and the previous row
                    self._board[(i, from_pos[1])] = self._board[(
                        i - 1, from_pos[1])]
                # move the piece up
                self._board[(0, from_pos[1])] = piece
            # if the player wants to slide it downward
            elif slide == Move.BOTTOM:
                # for each row starting from the row of the piece and going downward
                for i in range(from_pos[0], self._board.shape[0] - 1, 1):
                    # copy the value contained in the same column and the following row
                    self._board[(i, from_pos[1])] = self._board[(
                        i + 1, from_pos[1])]
                # move the piece down
                self._board[(self._board.shape[0] - 1, from_pos[1])] = piece
        return acceptable

## MyGame definition
***MyGame*** class is a subclass of *Game*: it inherits its methods and attributes and it contains also new useful methods.

In [ ]:
class MyGame(Game):
    """
    This class is used throughout the project instead of the Game class, as discussed with the professor, to adapt it to our needs.
    In this class we:
        - Override the __hash__ and __eq__ method to make the Game object hashable, and therefore usable as key in a dictionary;
        - Implement the is_valid method, which is used to check if a move is valid without modifying the current game, by applying the move function on a copy of the game;
        - Expose the move method to use it in a cleaner way throughout the project.
    """

    def __init__(self) -> None:
        super().__init__()

    def __str__(self) -> str:
        board = self.get_board()
        x_position = []
        o_position = []

        for i in range(board.shape[0]):
            for j in range(board.shape[1]):
                if board[i][j] == 1:
                    x_position.append((i, j))
                elif board[i][j] == 0:
                    o_position.append((i, j))

        str_x = "X: "
        for x in x_position:
            str_x += f"({x[0]},{x[1]}),"

        str_o = " O: "
        for o in o_position:
            str_o += f"({o[0]},{o[1]}),"

        return str_x + str_o


    def print(self):

        """
        IT OVERRIDES GAME'S print() METHOD

        Prints the board. '-' are neutral pieces, 'O' are pieces of player 0, 'X' pieces of player 1
        """

        board = [["" for _ in range(BOARD_SIZE)] for _ in range(BOARD_SIZE)]
        for x in range(self._board.shape[0]):
            for y in range(self._board.shape[1]):
                if self._board[x][y] == -1:
                    board[x][y] = "-"
                elif self._board[x][y] == 0:
                    board[x][y] = "O"
                else:
                    board[x][y] = "X"
        for x in board:
            print(x)

    def set_board(self, board):
        self._board = board


    def is_valid(self, from_pos: tuple[int, int], slide: Move, player_id: int) -> bool:
        cp = deepcopy(self)
        return cp._Game__move(from_pos, slide, player_id)

    def move(self, from_pos: tuple[int, int], slide: Move, player_id: int) -> bool:
        """
        Just to call __move() method from Game class; the method is private, but in MyGame we use it as public
        """
        return self._Game__move(from_pos, slide, player_id)

# TESTER RULE-ACTIONS

In [ ]:
# Test rule 8
g = MyGame()
g.current_player_idx = 0
print('current_player_idx: ', g.current_player_idx)

# secondary diagonal
"""
g.set_board(np.array([[ 1, 1, 1, 0, 1],
        [ 1,  0, -1, 1, 0],
        [ 0, -1, 1, 0, 0],
        [ -1, 1, 0, 0, 0],
        [ -1, 0, 1, 0, -1]]))

"""

# main diagonal
g.set_board(np.array([[ -1, 1, 1, 0, 1],
        [ 0,  1, -1, 1, 0],
        [ 0, -1, 1, 0, 0],
        [ -1, 1, 0, 1, 0],
        [ -1, 0, 1, 1, 1]]))



#print(g.get_board()[0,3]) # [row, column] -> 0

#g.set_board(np.array([[1,0,1,1,0], [1,0,1,1,0], [1,1,-1,0,0], [-1,0,0,1,1], [0,0,1,0,0]]))

"""
['X', '-', '-', 'O', 'O']
['X', '-', '-', '-', 'X']
['O', 'O', 'O', 'O', '-']
['O', '-', 'X', '-', '-']
['X', 'O', 'O', '-', 'X']
"""
#g.set_board(np.array([[1,-1,-1,0,0], [1,-1,-1,-1,1], [0,0,0,0,-1], [0,-1,1,-1,-1], [1,0,0,-1,1]]))

"""
['O', 'X', 'O', 'O', 'O']
['X', '-', '-', '-', 'X']
['X', 'O', 'O', '-', 'O']
['O', '-', 'X', '-', '-']
['X', 'O', 'O', 'X', 'X']
"""
#g.set_board(np.array([[0,1,0,0,0], [1,-1,-1,-1,1], [1,0,0,-1,0], [0,-1,1,-1,-1], [1,0,0,1,1]]))
#rule_that_you_want = 10#11
"""
['X', 'X', 'O', 'O', '-']
['O', 'X', '-', '-', 'X']
['O', '-', '-', 'O', 'X']
['O', '-', '-', 'X', 'O']
['X', 'O', 'O', 'X', 'X']
"""
#g.set_board(np.array([[1,1,0,0,-1], [0,1,-1,-1,1], [0,-1,-1,0,1], [0,-1,-1,1,0], [1,0,0,1,1]]))

rule_that_you_want = 8 # Put here the number of the rule that you want
associated_action = copy(rule_that_you_want)
rule = CONDITIONS[rule_that_you_want - 1]
action = ACTIONS[associated_action -1]
print('action: ', action)
result = rule(g)


print(result)
print("BEFORE board situation:\n")
g.print()
if result:
    from_pos, move = action[0](g)
    print('choosen action --> from_pos:', from_pos, 'move: ', move)
    g.move(from_pos, move, g.current_player_idx)

    print('AFTER board_situation: \n')
    g.print()

## Players definition

In [ ]:
class RandomPlayer(Player):
    def __init__(self) -> None:
        super().__init__()

    def make_move(self, game: "MyGame") -> tuple[tuple[int, int], Move]:
        from_pos = (random.randint(0, 4), random.randint(0, 4)) # (Col, Row)
        move = random.choice([Move.TOP, Move.BOTTOM, Move.LEFT, Move.RIGHT])
        return from_pos, move

class RulesBasedPlayer(Player):
    def __init__(self, λ = 23, initial_σ = 0.30, final_σ = 0.12) -> None:
        super().__init__()
        self.lamb = λ
        self.initial_sigma = initial_σ
        self.final_sigma = final_σ
        self.sigma = self.initial_sigma
        #self.initial_weights = [1/NUM_OF_RULES for _ in range(NUM_OF_RULES)]  # [1/14, 1/14 ...... 1/14]
        self.initial_weights = [random.uniform(-0.10, 0.10) for _ in range(NUM_OF_RULES)]
        self.initial_weights[7] = random.uniform(0.2, 0.7) # Considering 8 rules, the bad rules is in index 5 --> i want to se this positive value that goes down
        self.initial_weights = self.softmax(self.initial_weights)
        self.best_solution = self.initial_weights   # Best solution will be upload with the ES_1_plus_lambda()


        if os.path.exists(TRAINING_FILE_NAME):
            with open(TRAINING_FILE_NAME, 'rb') as file:
                print('file of training exists, load it')
                self.set_of_rules = pickle.load(file)
        else:
            self.set_of_rules =  self.ES_1_plus_lambda()


    def set_dictionary(self, list_of_weights):

        for index, (action, _) in enumerate(ACTIONS):
            weight = list_of_weights[index]
            ACTIONS[index] = (action, weight)

        # Dictionary self.set_of_rules: key = condition - value = (action, weight)
        self.set_of_rules = {rule: (action, weight) for rule, (action,weight) in zip(CONDITIONS, ACTIONS)}  # Creating a dictionary by combining the two lists 'conditions' and 'actions'

        return self.set_of_rules


    def ES_1_plus_lambda(self):

        eval_epoch = []
        history = []
        score_best_attual_solution = self.fitness(self.best_solution)
        counter = 0
        played_epochs = EPISODES_TRAINING

        for game in range(EPISODES_TRAINING):
            evals = []

            # Offspring will be an array of arrays. Each array is an individual, so  a fixed lenght array of weights
            offspring = [self.softmax((np.random.normal(loc=0, scale=self.sigma, size=NUM_OF_RULES) + self.best_solution)) for _ in range(self.lamb)]

            #I have to evaluate the fitness for each individual
            for weights_array in offspring:
                evals.append(self.fitness(weights_array))

            current_solution = offspring[np.argmax(evals)]

            score_current_solution = max(evals)

            eval_epoch.append(score_current_solution)

            if score_best_attual_solution < score_current_solution:
                self.best_solution = np.copy(current_solution)
                score_best_attual_solution = score_current_solution
                history.append((game, score_best_attual_solution)) # For seeing how the fintess increase
                counter = 0
            else:
                counter += 1
                if counter >= TOLERANCE_EPOCHS:  # steady-state break condition: TOLERANCE_EPOCHS
                    print(f"Break at epoch n.{game +1 } because the fitness doesn't improve!")
                    print(f"Result epoch {game +1}:\n Best_set_of_weights: \n{self.best_solution} \n Sum of weights: \n{sum(self.best_solution)} \n Best_fitness: {score_best_attual_solution}")
                    played_epochs = game+1
                    break

            self.sigma = max(self.final_sigma, self.initial_sigma - (self.initial_sigma- self.final_sigma) * (game / EPISODES_TRAINING))

            print(f"Result epoch {game +1}:\n Best_set_of_weights: \n{self.best_solution} \n Sum of weights: \n{sum(self.best_solution)} \n Best_fitness: {score_best_attual_solution}")

        self.set_of_rules = self.set_dictionary(self.best_solution)

        # Store the training
        file = open(TRAINING_FILE_NAME, 'wb')
        pickle.dump(self.set_of_rules, file)
        file.close()


        # Fitness History
        if len(history) != 0:
            x, y = zip(*history)

            plt.scatter(x, y)

            plt.xlabel('epoch')
            plt.ylabel('fitness')


            plt.title('fitness over epochs')

            plt.show()

        # Plot: solutions over the epochs
        plt.plot(list(range(1, played_epochs+1)), eval_epoch)
        plt.xlabel('epoch')
        plt.ylabel('fitness')
        plt.grid()
        plt.title('Fitness over epochs')

        plt.show()

        return self.set_of_rules

    def softmax(self, weights):
        exp_weights = np.exp(weights)
        probabilities = exp_weights / np.sum(exp_weights)
        return probabilities


    def fitness(self, weights_array):

        available = []
        # Player1 plays choosing an anction thanks to the adaptive function
        # Player2 random player

        self.set_of_rules = self.set_dictionary(weights_array)

        counter1 = 0
        counter2 = 0
        for fitness_game in range (EPISODES_TRAINING_FITNESS):
            game = MyGame()
            winner = -1

            while winner < 0:
                game.current_player_idx +=1
                game.current_player_idx %= 2 # We consider 2 players

                available = self.get_possible_moves(game)

                if game.current_player_idx == 1: # Random move if player=1 is playing
                    from_pos, move = choice(available)
                    game.move(from_pos, move, game.current_player_idx)
                    winner = game.check_winner()

                else: # If player = 0  is playing

                    g = deepcopy(game)
                    from_pos, move = self.adaptive(g)
                    game.move(from_pos, move, game.current_player_idx)
                    winner = game.check_winner()


            if winner == 0: # agent
                counter1 += 1
            if winner == 1: # opponent
                counter2 += 1

        score = counter1/EPISODES_TRAINING_FITNESS
        return score


    def get_possible_moves(self, game: "MyGame") -> list[tuple[tuple[int, int], Move]]:
        possible_moves = []
        for x in range(BOARD_SIZE):
            for y in range(BOARD_SIZE):
                if x>=1 and x<BOARD_SIZE-1 and y>=1 and y<BOARD_SIZE-1: # to skip internal cubes
                    continue
                for move in [Move.TOP, Move.BOTTOM, Move.LEFT, Move.RIGHT]:
                    if game.is_valid((x, y), move, game.current_player_idx):
                        possible_moves.append(((x, y), move))
        return possible_moves


    # the voting mechanism is a sort of roulette wheel.
    def voting(self, list_of_actions):


        total_weight = sum(weight for _, weight in list_of_actions)

        #the sum has to be equal to one
        normalized_weights = [weight / total_weight for _, weight in list_of_actions]

        random_value = random.uniform(0, 1)

        current_weight_sum = 0
        for index, normalized_weight in enumerate(normalized_weights):
            current_weight_sum += normalized_weight
            if current_weight_sum >= random_value:
                return list_of_actions[index][0]  #list_of_actions[index][0] is the element 0 in the tuple (action, 0) --> action has to be in the form from_pos, MOVE



    def test_conditions(self, game: "MyGame", dict_rules):
        active_actions = []

        for condition, (action, weight) in dict_rules.items():
            result = condition(game)
            if result:
                active_actions.append((action, weight))

        return active_actions


    def adaptive(self, game: 'MyGame'):

        '''
          check rules active and return the mappend actions
        '''
        active_actions = self.test_conditions(game, self.set_of_rules)
        choosen_action = self.voting(active_actions)
        from_pos, move = choosen_action(game)
        return from_pos, move

    def make_move(self, game: "MyGame") -> tuple[tuple[int, int], Move]:

        '''
            It plays with the best configuration of weights
        '''

        active_actions = self.test_conditions(game, self.set_of_rules)
        choosen_action = self.voting(active_actions)
        from_pos, move = choosen_action(game)
        return from_pos, move

## Draw_pie_chart


In [ ]:
def draw_pie_chart(win_rate, loss_rate, draw_rate, title):
    # Define data
    data = [win_rate, loss_rate, draw_rate]
    labels = ["Wins", "Losses", "Draws"]
    colors = ["#29F05F", "#EC3954", "#4570F8"]

    # Create a pie chart
    fig, ax = plt.subplots()
    ax.pie(data, labels=labels, colors=colors, autopct="%1.1f%%")
    ax.set_title(title)

    # Show the chart
    plt.show()

### Training
We make here the training of ES_1_plus_lambda player. Once trained the ES_1_plus_lambda will play with the same configuration of weights (the last and best configuration)

In [ ]:
player1 = RulesBasedPlayer()

### Testing
We test here the performances of our agent for EPISODES_GAME matches.

In [ ]:
counter1 = 0
counter2 = 0
start_time = time.time()

for _ in tqdm(range(EPISODES_GAME)):
    g = MyGame()
    g.print()
    print()

    player2 = RandomPlayer()
    winner = g.play(player1, player2) # Our agent starts first
    if winner == 0: # Agent
        counter1 += 1
    if winner == 1: # opponent
        counter2 += 1

    g.print()
    print(f"Winner: Player {winner}")

end_time = time.time()
elapsed_time = end_time - start_time

print("Win: ", counter1, "/", EPISODES_GAME)
print("Losses ", counter2, "/", EPISODES_GAME)
print("Ties: ", EPISODES_GAME - counter1 - counter2, "/", EPISODES_GAME)
print("Time for: ", EPISODES_GAME, "--> ", elapsed_time)
draw_pie_chart(
    counter1,
    counter2,
    EPISODES_GAME - counter1 - counter2,
    f"Results with {EPISODES_GAME} games",
)